In [1]:
from transformers import AutoTokenizer, AutoModel
import json
import os
import pickle
from asyncio import sleep
from queue import Queue

import torch
import torch.nn as nn
from tqdm import tqdm

from chatglm_6b_split_server import ChatGLMTokenizer, ChatGLMConfig, ChatGLMForConditionalGeneration
import os
from IPython.display import display, Markdown, clear_output

tokenizer_config_path = os.path.join("..","..","..","checkpoints","chatglm-6b", 'tokenizer_config.json')
model_config_path = os.path.join("..","..","..","checkpoints","chatglm-6b", 'config.json')
token_text_path = os.path.join("..","..","..","checkpoints","chatglm-6b", "ice_text.model")
model_dir = os.path.join("..","..","..","checkpoints","chatglm-6b")
model_state_dict_file_num = 8

def load_model():
    tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
    model = AutoModel.from_pretrained(model_dir, trust_remote_code=True).half().cuda()
    model = model.eval()
    return model, tokenizer

/home/project-1/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-12 11:52:30,103] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model, tokenizer = load_model()
print(model)

Loading checkpoint shards:  50%|█████     | 4/8 [00:03<00:03,  1.03it/s]

Loading checkpoint shards: 100%|██████████| 8/8 [00:06<00:00,  1.17it/s]


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4096, out_features=130528, bias=False)
)


In [3]:
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

The dtype of attention mask (torch.int64) is not bool


你好👋！我是人工智能助手 ChatGLM-6B，很高兴见到你，欢迎问我任何问题。


In [4]:
prompt = "如何制作宫保鸡丁"
for response, history in model.stream_chat(
        tokenizer, prompt, history=[]):
    clear_output(wait=True)
    display(Markdown(response))

制作宫保鸡丁的具体方法如下：

材料：
- 鸡胸肉 200 克
- 干辣椒 10 个
- 葱姜蒜适量
- 料酒、盐、糖、生抽、醋、淀粉、油适量

步骤：

1. 鸡胸肉切成丁，用料酒、盐、淀粉腌制 10 分钟左右；

2. 干辣椒剪成段，葱姜蒜切末备用；

3. 热锅凉油，将腌制好的鸡丁放入锅中煸炒至变色；

4. 盛出备用；

5. 锅中留底油，放入葱姜蒜末煸炒出香味，加入干辣椒煸炒至香；

6. 将炒好的干辣椒和鸡丁放入锅中，加入适量的生抽、醋、糖、盐、料酒，翻炒均匀；

7. 加入适量的水，焖煮 3 分钟左右，收汁即可。

温馨提示：

1. 干辣椒可以根据个人口味适量调整；

2. 鸡丁煸炒至变色即可，过度煸炒会影响口感；

3. 焖煮的时间可以根据自己的口感调整，如果喜欢口感脆爽，可以适当延长焖煮时间。